In [1]:
import numpy as np
from scipy import sparse


M = sparse.dok_matrix((0,0))

M.resize(2,2)
M[1,1] = 1
print(M.todense())

print("\nagrego nodo:")

M.resize(3,3)
M[1,2] = 5
print(M.todense())

max(M.get_shape())

validas = [1,2]
M[:,validas][validas,:].todense()

[[0. 0.]
 [0. 1.]]

agrego nodo:
[[0. 0. 0.]
 [0. 1. 5.]
 [0. 0. 0.]]


matrix([[1., 5.],
        [0., 0.]])

In [2]:
from Grafo import Grafo

In [3]:
l = ['luis', 'pepe']
d = {i:v for v,i in enumerate(l)}
d['luis']

0

In [4]:
grafito = Grafo()

grafito.agregar_nodo("pepe")
grafito.agregar_nodo("luis")
grafito.agregar_nodo("solito")
grafito.agregar_arista("pepe","luis",3)
grafito.agregar_arista("luis","pepe",-3)
print(grafito)

grafito.agregar_nodo("amiguis1")
grafito.agregar_nodo("amiguis2")
grafito.agregar_arista("amiguis1","amiguis2",no_dirigido=True)
print(grafito)

TypeError: 'module' object is not callable

In [ ]:
grafito.eliminar_arista("amiguis1","amiguis2")
print(grafito)

grafito.eliminar_nodo("luis")
print(grafito)

In [ ]:
x=['holis','gaston','gato']

if 'pepe' in x:
    print(3)
else:
    print(2)
    
x.index('gaston')

In [ ]:
x

In [ ]:
m={"pepe":1, "gaston":4}
m["gaston"]

In [ ]:
m["popo"]=-1

In [ ]:
m

In [ ]:
list(m.values())

In [ ]:
0.0==0

In [ ]:
ts=grafito._mat_ady[:,0]

In [ ]:
ts

In [ ]:
grafito._mat_ady[:,0]=0

In [ ]:
grafito._mat_ady.todense()